In [6]:
import cv2

import numpy as np
import keras

from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Add, concatenate, AveragePooling2D, Lambda, GlobalMaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D, GaussianNoise
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras import activations
import os
from matplotlib import pyplot as plt
from random import shuffle
import tensorflow as tf


In [2]:
random_seed=305


np.random.seed(random_seed)
tf.set_random_seed(random_seed)

In [3]:
defect_class = "BIN"

modelname = defect_class + "_Single_BIN_Crop.hdf5."
train_path = './Data/BIN_CROP_train.npz'
test_path = './Data/BIN_CROP_test.npz'

saved_trainset = np.load(train_path)
saved_testset = np.load(test_path)

X_train_data = saved_trainset['x']
train_data_y = saved_trainset['y']

X_train_data = X_train_data/255

n_data = X_train_data.shape[0]
indices = np.arange(n_data)
np.random.shuffle(indices)

train_index = indices[int(n_data * 0.2):n_data+1]
valid_index = indices[0:int(n_data * 0.2)]

X_train = X_train_data[train_index]
X_valid = X_train_data[valid_index]
train_y = train_data_y[train_index]
valid_y = train_data_y[valid_index]



X_test = saved_testset['x']
test_y = saved_testset['y']

X_test=X_test/255

print(X_train.shape)
print(train_y.shape)
print(X_test.shape)
print(test_y.shape)
print(X_valid.shape)
print(valid_y.shape)

(1920, 256, 256, 1)
(1920, 4)
(476, 256, 256, 1)
(476, 4)
(480, 256, 256, 1)
(480, 4)


In [4]:
train_datagen = ImageDataGenerator(
    zoom_range=0.3, 
    horizontal_flip=True, 
    vertical_flip=True,
    rotation_range=90)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, train_y, batch_size = 50)
test_generator = test_datagen.flow(X_test, test_y, batch_size = 30)
model = Sequential()
model.add(Conv2D(64, (7, 7), strides = 2, activation='elu', padding='same', input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='elu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='elu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax', name='preds'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      3200      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)       0         
__________

In [5]:
plt.rcParams['figure.figsize'] = (20, 9)

sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,  metrics=['accuracy'])

hist = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=300,
        verbose=1,
        validation_data=test_generator,
        validation_steps=30, 
        callbacks=None)

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

plt.show()